In [3]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import shutil
import os
import csv
import sys

import json
import datetime
import pandas as pd
import numpy as np
import talib as ta
import math

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(hours=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

def logarism(x):
    x = abs(x)
    return math.log(x)


def get_now_data(time, period_hours, money, foot, back_hours = 0, times = 0):
    global BACK
    all_data = []
    #time = time.replace(hour=7, hour=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    NY = datetime.timedelta(hours=-14)
    back = datetime.timedelta(hours=-back_hours)
    
    force = datetime.timedelta(hours=-BACK)
    
    date_to = time + NY + datetime.timedelta(seconds=-1) + back + force

    date_from = date_to + datetime.timedelta(hours=-period_hours)
#     for i in range(times+1):    
    ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

    date_to = date_from
    date_from = date_to + datetime.timedelta(hours=-period_hours)
#         print(date_from, date_to)

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

#     print(date_from, date_to)
#     print(df)
    
    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    
    df['Volume'] = df['Volume'].astype('double')
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    df['High'] = df['High'].astype('double')
    df['Low'] = df['Low'].astype('double')
    
    
    close = np.array(df["Close"]).astype(float)
#     #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる
#     #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df["SMA2"]= ta.EMA(close, timeperiod=2) / close
#     df["SMA3"]= ta.SMA(close, timeperiod=3) / close
#     df["SMA4"]= ta.SMA(close, timeperiod=4) / close
#     df["SMA5"]= ta.EMA(close, timeperiod=5)# / close
#     df["SMA6"]= ta.SMA(close, timeperiod=6) / close
#     df["SMA7"]= ta.SMA(close, timeperiod=7) / close
#     df["SMA8"]= ta.SMA(close, timeperiod=8) / close
#     df["SMA9"]= ta.SMA(close, timeperiod=9) / close
#     df["SMA10"]= ta.EMA(close, timeperiod=10) / close
    df["RSI"] = ta.RSI(close, timeperiod=9) / close
    #ボリンジャーバンド Main 
    upper, middle, lower = ta.BBANDS(close, timeperiod=10, nbdevup=2, nbdevdn=2)
    df["BBANDS+σ"] = upper / close * 1000
    df["BBANDS-σ"] = lower / close * 1000
    
    #df['Rate']=(df['Close']-df['Open'])/df['Open']
    
#     df['Open'] = df['Open'].apply(lambda x: logarism(x))
#     df['Close'] = df['Close'].apply(lambda x: logarism(x))
    
    df["SMA2"]= df["SMA2"].apply(lambda x: logarism(x)) * 1000
#     df["SMA3"]= df["SMA3"].apply(lambda x: logarism(x)) * 1000
#     df["SMA4"]= df["SMA4"].apply(lambda x: logarism(x)) * 1000
#     df["SMA5"]= df["SMA5"].apply(lambda x: logarism(x)) * 1000
#     df["SMA6"]= df["SMA6"].apply(lambda x: logarism(x)) * 1000
#     df["SMA7"]= df["SMA7"].apply(lambda x: logarism(x)) * 1000
#     df["SMA8"]= df["SMA8"].apply(lambda x: logarism(x)) * 1000
#     df["SMA9"]= df["SMA9"].apply(lambda x: logarism(x)) * 1000
#     df["SMA10"]= df["SMA10"].apply(lambda x: logarism(x)) * 1000
    
    df["MACD"], _ , _= ta.MACD(close, fastperiod=3, slowperiod=18, signalperiod=9)/close
    
    
#     df['Rate']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
#     df['Rate'] = df['Rate']*1000
#     df['true']=(df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')
    
    df['class'] = ((df['Close'].astype('double')-df['Open'].astype('double'))/df['Open'].astype('double')).apply(lambda x: classify_two(x))
    df['class'] = df['class'].shift(-1)
    
#     print(df.head(1))
    df_next = df.tail(1)
    
    df = df.dropna()
    
#     df['Open'] = df['Open']/df['Close']
    
    df = df.drop('Open', axis=1)
    df = df.drop('High', axis=1)
    df = df.drop('Low', axis=1)
    df = df.drop('Close', axis=1)
    df = df.drop('Volume', axis=1)
    
    df = df.set_index('Datetime')
    
    return df, df_next



def get_csv(back_hours, hours, money):
    global MULTI
    time = datetime.datetime.now() 
    
    df, df_next = get_now_data(time, period_hours=hours, money=money, foot = FOOT , back_hours=back_hours, times=MULTI)
    date_to = (time+datetime.timedelta(hours=-back_hours)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(hours=-hours)+datetime.timedelta(hours=-back_hours)).strftime("%Y-%m-%d")
    df.to_csv("csv/"+str(money)+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to +"_"+str(hours*MULTI)+'.csv', end='", ')
    
    return df

def get_moneys_csv(back_hours, hours, moneylist):
    csvlist = []
    
    time = datetime.datetime.now()
    date_to = (time+datetime.timedelta(hours=-back_hours)+datetime.timedelta(seconds=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(hours=-hours)+datetime.timedelta(hours=-back_hours)).strftime("%Y-%m-%d")
    
    for i in range (len(moneylist)):
        get_csv(back_hours, hours, money=moneylist[i])
        csvlist.extend([str(moneylist[i])+"_"+ date_from + "_" + date_to +"_"+str(hours)+'.csv'])
    
    return csvlist
        
def classify_two(x):
        if x<0:
            return -1
        elif 0<=x:
            return 1
        
def classify_random(x):
        ram = random.randint(0,1)
        if ram==0:
            return -1
        elif ram==1:
            return 1
        

def get_int_csv(moneylist):
    global TERM, DNUM
    shutil.rmtree('csv')
    os.mkdir('csv')
    array = get_moneys_csv(back_hours=TERM, hours=DNUM, moneylist = moneylist)
    return array

def make_bins(max_num, min_num, bin_num):
    global DIVIDE
    delta = max_num-min_num
    delta = delta/(DIVIDE-1)
    bins = []
    povit = min_num
    for i in range(DIVIDE-1):
        povit += delta
        bins.extend([povit])
#     print(bins)
    return bins

def digitize_data(df):
    bin_num = len(df)
    for i in range(len(df.columns)):
        col = df.columns.values.tolist()[i]
        if col == "class":
            continue
        elif col == "true":
            continue
        else:
            max_num = df[col].max()
            min_num = df[col].min()
#             max_num = df[col].quantile(0.75)
#             min_num = df[col].quantile(0.25)
            df[col] = np.digitize(df[col], make_bins(max_num, min_num, bin_num))
    return df

def grider(dataname):
    df = pd.read_csv("csv/"+dataname, index_col='Datetime')
    df = digitize_data(df)
#     print(df)
    return df
    
def make_table(line, money):
    global DIVIDE
    count = 0
    df = grider(money)
    print("Table data")
    print(df.head(1).index.values, end=' to ')
    print(df.tail(1).index.values)

#     table = np.zeros((DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE))
    table = np.zeros((DIVIDE,DIVIDE,DIVIDE,DIVIDE,DIVIDE))
    for y in range(len(df)):
        temp = []
        for x in range(len(df.columns)-1):
            temp.extend([df.iat[y,x]])
#         table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] += int(df.iat[y,len(df.columns)-1])
        table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]] += int(df.iat[y,len(df.columns)-1])
        count += 1
    print("Beyond LINE: ", end='')
    print(np.count_nonzero(table>line)/np.count_nonzero(table!=0))
    print("table count: "+str(count))
    return table

def buy_test(table, money):
    global FOOT, TERM, LINE, POSSESTION, PAY, PAYOUT, WIN, LOSE, COUNT
    df_now, df_next = get_now_data(datetime.datetime.now(), TERM, money, FOOT, back_hours = 0, times = 0)
#     df_now['class'] = df_now['class'].shift(1)
    df_now = digitize_data(df_now)
    df_now = df_now.dropna()
    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    count = [0,0,0]
    win = 0
    lose = 0
    glaph = []
    pos = POSSESTION
    history = []
    for y in range(len(df_now)):
        temp = []
        for x in range(len(df_now.columns)-1):
            temp.extend([df_now.iat[y,x]])
#         value = table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]]
        value = table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]]
        if value>=LINE: #HIGH予想
            count[0] += 1
            pos -= PAY
            if(int(df_now.iat[y,len(df_now.columns)-1])>0): #実際HIGHなら
                pos += PAY*PAYOUT
                win += 1
                history.extend([0.9])
            else:
                lose += 1
                history.extend([-1])
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]]
                table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]]
                continue
        elif value<=-LINE: #LOW予想
            count[1] += 1
            pos -= PAY
            if(int(df_now.iat[y,len(df_now.columns)-1])<0): #実際LOWなら
                pos += PAY*PAYOUT
                win += 1
                history.extend([0.9])
            else:
                lose += 1
                history.extend([-1])
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]][temp[5]]
                table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]] = 0 #never use this cell
#                 table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]] = -1*table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]]
                continue
        else:
            count[2] += 1
            history.extend([0])
        glaph.extend([pos])
    print("HIGH/LOW/STAY")
    print(count)
    print("win: "+str(win))
    print("lose: "+str(lose))
    
    WIN += win
    LOSE += lose
    COUNT += sum(count)
    
    if(win+lose==0):
        print("No Trade")
    else:
        print("Rate: "+str(win/(win+lose)))
    print("Posesstion: "+str(pos))
    POSSESTION += pos - POSSESTION
    
#     POSSESTION = 10000
    
#     if(POSSESTION < 0):
#         sys.exit()
#     plt.plot(list(range(0,len(glaph))), glaph)
#     plt.show()

#     for k in range(len(df_next.columns)-1):
#             temp.extend([df_next.iat[1,k]])
#     value = table[temp[0]][temp[1]][temp[2]][temp[3]][temp[4]]
#     print(value)
    return win,lose, history

def main():
    global LINE, POSSESTION
    array = get_int_csv(MONEYS)
    w = 0
    l = 0
    for i in range(len(array)):
        print("")
        print("-----------------------")
        print(MONEYS[i])
        table = make_table(LINE, array[i])
#         print(table)
        win, lose, history = buy_test(table, MONEYS[i])
        w += win
        l += lose
#         with open('result.csv', 'a') as f:
#             writer = csv.writer(f)
#             writer.writerow([MONEYS[i], history])
    print("RESULT: "+str(POSSESTION))
    RESULT.extend([POSSESTION])
    with open('result.csv', 'a') as f:
        writer = csv.writer(f)
#         writer.writerow([money, pos])
        if(w+l!=0):
            writer.writerow([POSSESTION, w/(w+l)])
        else:
            writer.writerow([POSSESTION])
    

FOOT = 'H1'
DNUM = 100
MULTI = 1  #datanum = DNUM * MULTI
DIVIDE = 2  #GRIDの数
LINE = 1  #トレードのライン(LINE以上，以下の時エントリー)
TERM = 100  #テスト期間
PAY = 1000  #支払額
PAYOUT = 1.9 #ペイアウト率
POSSESTION = 10000 #所持金
BACK = 0
RESULT = []
WIN = 0
LOSE = 0
COUNT = 0

MONEYS = [
        'AUD_JPY',
    
#         'AUD_NZD',
#         'AUD_USD',
#         'CAD_JPY',
#         'CHF_JPY',
#         'EUR_AUD',
#         'EUR_GBP',
    
        'EUR_JPY',
        'EUR_USD',
    
#         'GBP_AUD',
    
        'GBP_JPY',
    
#         'GBP_USD',
#         'NZD_JPY',
#         'NZD_USD',
#         'USD_CAD',
#         'USD_CHF',
        'USD_JPY'
    ]

# MONEYS = [
#         'AUD_JPY',
#         'AUD_NZD',
#         'AUD_USD',
#         'CAD_JPY',
#         'CHF_JPY',
#         'EUR_AUD',
#         'EUR_GBP',
#         'EUR_JPY',
#         'EUR_USD',
#         'GBP_AUD',
#         'GBP_JPY',
#         'GBP_USD',
#         'NZD_JPY',
#         'NZD_USD',
#         'USD_CAD',
#         'USD_CHF',
#         'USD_JPY'
#     ]

# np.set_printoptions(threshold=np.inf)

for x in range(1000):
    print("")
    print("TEST: "+ str(x))
    main()
    BACK += TERM
    print("Trade/Trade chance: "+str(WIN+LOSE)+"/"+str(COUNT))
    print("Trade rate: "+str((WIN+LOSE)/COUNT))
    print("Rate: "+str(WIN/(WIN+LOSE)))

# DNUM = 100
# DIVIDE = 2
# BEST = 0
# LINE = 1
# for y in range(5):
#     for i in range(10):
#         for j in range(7):
#             print("DNUM:"+str(DNUM)+" /DIVIDE:"+str(DIVIDE)+ " /LINE:" +str(LINE)+" /BEST:"+str(BEST))
#             for x in range(100):
#                 print("")
#                 print("TEST: "+ str(x))
#                 main()
#                 BACK += TERM
#                 if(LOSE==0 and WIN==0):
#                     LOSE = 1
#                 print("Trade/Trade chance: "+str(WIN+LOSE)+"/"+str(COUNT))
#                 print("Trade rate: "+str((WIN+LOSE)/COUNT))
#                 print("Rate: "+str(WIN/(WIN+LOSE)))
#                 if(POSSESTION<0):
#                     break
#             with open('result.csv', 'a') as f:
#                     writer = csv.writer(f)
#                     if(BEST<WIN/(WIN+LOSE)):
#                         BEST = WIN/(WIN+LOSE)
#                         writer.writerow(["BEST", DNUM, DIVIDE, LINE, WIN/(WIN+LOSE), POSSESTION])
#                     else:
#                         writer.writerow([DNUM, DIVIDE, LINE, WIN/(WIN+LOSE), POSSESTION])
#             WIN = 0
#             LOSE = 0
#             POSSESTION = 10000
#             BACK = 0
#             COUNT = 0
#             DIVIDE += 1
#         DNUM+=100
#         DIVIDE = 2
#         BACK = 0
#     DNUM = 100
#     LINE += 1
        


TEST: 0
"AUD_JPY_2021-01-23_2021-01-27_100.csv", "EUR_JPY_2021-01-23_2021-01-27_100.csv", "EUR_USD_2021-01-23_2021-01-27_100.csv", "GBP_JPY_2021-01-23_2021-01-27_100.csv", "USD_JPY_2021-01-23_2021-01-27_100.csv", 
-----------------------
AUD_JPY
Table data
['2021-01-26 08:00'] to ['2021-01-27 10:00']
Beyond LINE: 0.0
table count: 27

test data
['2021-01-28 12:00'] to ['2021-01-30 05:00']
HIGH/LOW/STAY
[1, 0, 41]
win: 0
lose: 1
Rate: 0.0
Posesstion: 9000

-----------------------
EUR_JPY
Table data
['2021-01-26 08:00'] to ['2021-01-27 10:00']
Beyond LINE: 0.0
table count: 27

test data
['2021-01-28 12:00'] to ['2021-01-30 05:00']
HIGH/LOW/STAY
[0, 3, 39]
win: 1
lose: 2
Rate: 0.3333333333333333
Posesstion: 7900.0

-----------------------
EUR_USD
Table data
['2021-01-26 08:00'] to ['2021-01-27 10:00']
Beyond LINE: 0.0
table count: 27

test data
['2021-01-28 12:00'] to ['2021-01-30 05:00']
HIGH/LOW/STAY
[0, 0, 42]
win: 0
lose: 0
No Trade
Posesstion: 7900.0

-----------------------
GBP_JPY



test data
['2021-01-07 16:00'] to ['2021-01-09 05:00']
HIGH/LOW/STAY
[1, 0, 37]
win: 0
lose: 1
Rate: 0.0
Posesstion: 21200.0

-----------------------
EUR_JPY
Table data
['2021-01-05 08:00'] to ['2021-01-06 14:00']
Beyond LINE: 0.0
table count: 31

test data
['2021-01-07 16:00'] to ['2021-01-09 05:00']
HIGH/LOW/STAY
[0, 4, 34]
win: 3
lose: 1
Rate: 0.75
Posesstion: 22900.0

-----------------------
EUR_USD
Table data
['2021-01-05 08:00'] to ['2021-01-06 14:00']
Beyond LINE: 0.2
table count: 31

test data
['2021-01-07 16:00'] to ['2021-01-09 05:00']
HIGH/LOW/STAY
[1, 0, 37]
win: 0
lose: 1
Rate: 0.0
Posesstion: 21900.0

-----------------------
GBP_JPY
Table data
['2021-01-05 08:00'] to ['2021-01-06 14:00']
Beyond LINE: 0.25
table count: 31

test data
['2021-01-07 16:00'] to ['2021-01-09 05:00']
HIGH/LOW/STAY
[5, 1, 32]
win: 4
lose: 2
Rate: 0.6666666666666666
Posesstion: 23500.0

-----------------------
USD_JPY
Table data
['2021-01-05 08:00'] to ['2021-01-06 14:00']
Beyond LINE: 0.0
table c


test data
['2020-12-17 20:00'] to ['2020-12-19 05:00']
HIGH/LOW/STAY
[3, 2, 29]
win: 4
lose: 1
Rate: 0.8
Posesstion: 37000.0

-----------------------
GBP_JPY
Table data
['2020-12-15 08:00'] to ['2020-12-16 18:00']
Beyond LINE: 0.3333333333333333
table count: 35

test data
['2020-12-17 20:00'] to ['2020-12-19 05:00']
HIGH/LOW/STAY
[2, 0, 32]
win: 1
lose: 1
Rate: 0.5
Posesstion: 36900.0

-----------------------
USD_JPY
Table data
['2020-12-15 08:00'] to ['2020-12-16 18:00']
Beyond LINE: 0.0
table count: 35

test data
['2020-12-17 20:00'] to ['2020-12-19 05:00']
HIGH/LOW/STAY
[1, 3, 30]
win: 2
lose: 2
Rate: 0.5
Posesstion: 36700.0
RESULT: 36700.0
Trade/Trade chance: 169/2261
Trade rate: 0.07474568774878372
Rate: 0.6094674556213018

TEST: 11
"AUD_JPY_2021-01-23_2021-01-27_100.csv", "EUR_JPY_2021-01-23_2021-01-27_100.csv", "EUR_USD_2021-01-23_2021-01-27_100.csv", "GBP_JPY_2021-01-23_2021-01-27_100.csv", "USD_JPY_2021-01-23_2021-01-27_100.csv", 
-----------------------
AUD_JPY
Table data
['


test data
['2020-11-27 00:00'] to ['2020-11-28 05:00']
HIGH/LOW/STAY
[0, 2, 28]
win: 1
lose: 1
Rate: 0.5
Posesstion: 64500.0

-----------------------
USD_JPY
Table data
['2020-11-24 08:00'] to ['2020-11-25 22:00']
Beyond LINE: 0.25
table count: 39

test data
['2020-11-27 00:00'] to ['2020-11-28 05:00']
HIGH/LOW/STAY
[3, 0, 27]
win: 1
lose: 2
Rate: 0.3333333333333333
Posesstion: 63400.0
RESULT: 63400.0
Trade/Trade chance: 262/3406
Trade rate: 0.07692307692307693
Rate: 0.6335877862595419

TEST: 16
"AUD_JPY_2021-01-23_2021-01-27_100.csv", "EUR_JPY_2021-01-23_2021-01-27_100.csv", "EUR_USD_2021-01-23_2021-01-27_100.csv", "GBP_JPY_2021-01-23_2021-01-27_100.csv", "USD_JPY_2021-01-23_2021-01-27_100.csv", 
-----------------------
AUD_JPY
Table data
['2020-11-18 16:00'] to ['2020-11-21 05:00']
Beyond LINE: 0.0
table count: 62

test data
['2020-11-24 08:00'] to ['2020-11-25 22:00']
HIGH/LOW/STAY
[1, 0, 38]
win: 1
lose: 0
Rate: 1.0
Posesstion: 64300.0

-----------------------
EUR_JPY
Table data
[


test data
['2020-11-06 04:00'] to ['2020-11-07 06:00']
HIGH/LOW/STAY
[1, 2, 24]
win: 2
lose: 1
Rate: 0.6666666666666666
Posesstion: 68600.0
RESULT: 68600.0
Trade/Trade chance: 329/4556
Trade rate: 0.07221246707638279
Rate: 0.6200607902735562

TEST: 21
"AUD_JPY_2021-01-23_2021-01-27_100.csv", "EUR_JPY_2021-01-23_2021-01-27_100.csv", "EUR_USD_2021-01-23_2021-01-27_100.csv", "GBP_JPY_2021-01-23_2021-01-27_100.csv", "USD_JPY_2021-01-23_2021-01-27_100.csv", 
-----------------------
AUD_JPY
Table data
['2020-10-28 19:00'] to ['2020-10-31 04:00']
Beyond LINE: 0.2
table count: 58

test data
['2020-11-03 08:00'] to ['2020-11-05 02:00']
HIGH/LOW/STAY
[4, 2, 37]
win: 4
lose: 2
Rate: 0.6666666666666666
Posesstion: 70200.0

-----------------------
EUR_JPY
Table data
['2020-10-28 19:00'] to ['2020-10-31 04:00']
Beyond LINE: 0.0
table count: 58

test data
['2020-11-03 08:00'] to ['2020-11-05 02:00']
HIGH/LOW/STAY
[4, 0, 39]
win: 3
lose: 1
Rate: 0.75
Posesstion: 71900.0

-----------------------
EUR_U

...


test data
['2019-08-09 23:00'] to ['2019-08-13 01:00']
HIGH/LOW/STAY
[2, 2, 23]
win: 2
lose: 2
Rate: 0.5
Posesstion: 352600.0

-----------------------
EUR_JPY
Table data
['2019-08-06 07:00'] to ['2019-08-08 21:00']
Beyond LINE: 0.0
table count: 63

test data
['2019-08-09 23:00'] to ['2019-08-13 01:00']
HIGH/LOW/STAY
[0, 2, 25]
win: 1
lose: 1
Rate: 0.5
Posesstion: 352500.0

-----------------------
EUR_USD
Table data
['2019-08-06 07:00'] to ['2019-08-08 21:00']
Beyond LINE: 0.0
table count: 63

test data
['2019-08-09 23:00'] to ['2019-08-13 01:00']
HIGH/LOW/STAY
[0, 1, 26]
win: 0
lose: 1
Rate: 0.0
Posesstion: 351500.0

-----------------------
GBP_JPY
Table data
['2019-08-06 07:00'] to ['2019-08-08 21:00']
Beyond LINE: 0.25
table count: 63

test data
['2019-08-09 23:00'] to ['2019-08-13 01:00']
HIGH/LOW/STAY
[2, 0, 25]
win: 1
lose: 1
Rate: 0.5
Posesstion: 351400.0

-----------------------
USD_JPY
Table data
['2019-08-06 07:00'] to ['2019-08-08 21:00']
Beyond LINE: 0.0
table count: 63

te


test data
['2019-07-20 03:00'] to ['2019-07-23 05:00']
HIGH/LOW/STAY
[2, 1, 24]
win: 0
lose: 3
Rate: 0.0
Posesstion: 361200.0

-----------------------
EUR_USD
Table data
['2019-07-16 07:00'] to ['2019-07-19 01:00']
Beyond LINE: 0.25
table count: 67

test data
['2019-07-20 03:00'] to ['2019-07-23 05:00']
HIGH/LOW/STAY
[4, 2, 21]
win: 4
lose: 2
Rate: 0.6666666666666666
Posesstion: 362800.0

-----------------------
GBP_JPY
Table data
['2019-07-16 07:00'] to ['2019-07-19 01:00']
Beyond LINE: 0.25
table count: 67

test data
['2019-07-20 03:00'] to ['2019-07-23 05:00']
HIGH/LOW/STAY
[2, 2, 24]
win: 3
lose: 1
Rate: 0.75
Posesstion: 364500.0

-----------------------
USD_JPY
Table data
['2019-07-16 07:00'] to ['2019-07-19 01:00']
Beyond LINE: 0.0
table count: 67

test data
['2019-07-20 03:00'] to ['2019-07-23 05:00']
HIGH/LOW/STAY
[0, 0, 27]
win: 0
lose: 0
No Trade
Posesstion: 364500.0
RESULT: 364500.0
Trade/Trade chance: 2277/30576
Trade rate: 0.0744701726844584
Rate: 0.6082564778216952

TEST


test data
['2019-07-01 07:00'] to ['2019-07-02 09:00']
HIGH/LOW/STAY
[1, 5, 21]
win: 5
lose: 1
Rate: 0.8333333333333334
Posesstion: 373800.0

-----------------------
GBP_JPY
Table data
['2019-06-25 07:00'] to ['2019-06-28 05:00']
Beyond LINE: 0.0
table count: 71

test data
['2019-07-01 07:00'] to ['2019-07-02 09:00']
HIGH/LOW/STAY
[1, 4, 22]
win: 4
lose: 1
Rate: 0.8
Posesstion: 376400.0

-----------------------
USD_JPY
Table data
['2019-06-25 07:00'] to ['2019-06-28 05:00']
Beyond LINE: 0.25
table count: 71

test data
['2019-07-01 06:00'] to ['2019-07-02 09:00']
HIGH/LOW/STAY
[3, 0, 25]
win: 2
lose: 1
Rate: 0.6666666666666666
Posesstion: 377200.0
RESULT: 377200.0
Trade/Trade chance: 2365/31724
Trade rate: 0.07454923717059639
Rate: 0.6080338266384778

TEST: 140
"AUD_JPY_2021-01-23_2021-01-27_100.csv", "EUR_JPY_2021-01-23_2021-01-27_100.csv", "EUR_USD_2021-01-23_2021-01-27_100.csv", "GBP_JPY_2021-01-23_2021-01-27_100.csv", "USD_JPY_2021-01-23_2021-01-27_100.csv", 
----------------------


test data
['2019-06-10 11:00'] to ['2019-06-11 13:00']
HIGH/LOW/STAY
[2, 1, 24]
win: 2
lose: 1
Rate: 0.6666666666666666
Posesstion: 398800.0

-----------------------
USD_JPY
Table data
['2019-06-04 07:00'] to ['2019-06-07 09:00']
Beyond LINE: 0.2
table count: 75

test data
['2019-06-10 10:00'] to ['2019-06-11 13:00']
HIGH/LOW/STAY
[5, 2, 21]
win: 5
lose: 2
Rate: 0.7142857142857143
Posesstion: 401300.0
RESULT: 401300.0
Trade/Trade chance: 2453/32873
Trade rate: 0.07462050923250084
Rate: 0.610273134936812

TEST: 145
"AUD_JPY_2021-01-23_2021-01-27_100.csv", "EUR_JPY_2021-01-23_2021-01-27_100.csv", "EUR_USD_2021-01-23_2021-01-27_100.csv", "GBP_JPY_2021-01-23_2021-01-27_100.csv", "USD_JPY_2021-01-23_2021-01-27_100.csv", 
-----------------------
AUD_JPY
Table data
['2019-05-31 03:00'] to ['2019-06-01 05:00']
Beyond LINE: 0.0
table count: 27

test data
['2019-06-04 07:00'] to ['2019-06-07 09:00']
HIGH/LOW/STAY
[1, 4, 70]
win: 4
lose: 1
Rate: 0.8
Posesstion: 403900.0

-----------------------


KeyboardInterrupt: 

DNUM:500 /DIVIDE:6 /LINE:2
DNUM:600 /DIVIDE:6 /LINE:2

In [ ]:
OLD DATA

#DNUM: 300, range:10, 12/12~1/22
#not 0 falut
#0 reset 20500 rate:0.526...
#-1 antize 88200 rate:0.575...

#DNUM: 300, range:100, 2019/12/03~2021/1/22
#0 reset 396400 rate:0.526...
#-1 antize 971200 rate:0.590...


#DNUM: 300, range:100, 2019/12/03~2021/1/22 GRID=7
#-1 antize 409000 rate:0.5797...

#DNUM: 300, range:100, 2019/12/03~2021/1/22 GRID=6
#-1 antize 890400 rate:0.5936...

#DNUM: 300, range:100, 2019/12/03~2021/1/22 GRID=5
#-1 antize 2039900 rate:0.6183...

#DNUM: 300, range:100, 2019/12/03~2021/1/22 GRID=4
#-1 antize 3114100 rate:0.6091...

#DNUM: 300, range:100, 2019/12/03~2021/1/22 GRID=3
#-1 antize 11614400 rate:0.68851...

#DNUM: 100, range:100, 2019/12/03~2021/1/22 GRID=3
#-1 antize (2671400) rate:0.67243...

#DNUM: 200, range:100, 2019/12/03~2021/1/22 GRID=3
#-1 antize (3069000) rate:0.67419...

#DNUM: 400, range:100, 2019/12/03~2021/1/22 GRID=3
#-1 antize (2945100) rate:0.65800...

#DNUM: 300, range:1000, 2009/8/28~2021/1/22 GRID=3
#-1 antize (27751500) rate:0.65301...